<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2Fimage_segmentation-scar/Image_segmentation_Scar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set development enviroment

## Install

## Import

### Library

In [6]:
# Image processing
import cv2

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os

### Google drive mount

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/Models/GAN_Scar
!ls -al

/content/gdrive/MyDrive/Models/GAN_Scar
total 225
drwx------ 4 root root   4096 Aug  9 13:27 Dataset
-rw------- 1 root root 225811 Aug  9 14:00 Image_segmentation-Scar.ipynb


# Image segmentation

## Global variable

In [4]:
MODEL_PATH = "/content/gdrive/MyDrive/Models/GAN_Scar"

TRAIN_SET_PATH = "/Dataset/train"
TEST_SET_PATH = "/Dataset/test"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

## Class

## Function

In [11]:
def imshow_waitkey_enter(image):
    cv2_imshow(image)

    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

## Run

In [ ]:
# Set load image dir path
dirPath = MODEL_PATH + TRAIN_SET_PATH + IMAGES_PATH

# Load images
os.chdir(dirPath)
files = os.listdir(dirPath)

train_images = []
for f in files: 
    train_images.append(cv2.imread(f))

# Preprocess image
for image in train_images:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, ths_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_OTSU)